# Web Scraping

## Objective

Extract data from Web pages.

Extract weather information about downtown San Francisco from [this page](http://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168).

Combine the data into a Pandas DataFrame and analyze it.

## Web Overview

![alt text](http://i.imgur.com/fz99TlR.png)

The page has information about the extended forecast for the next week, including time of day, temperature, and a brief description of the conditions.

## Exploring page structure with Chrome DevTools

Start the developer tools in Chrome by clicking View -> Developer -> Developer Tools.

Discover that each forecast item (like “Tonight”, “Thursday”, and “Thursday Night”) is contained in a div with the class tombstone-container.

![alt text](http://i.imgur.com/DPVxqmf.png)

## Download the page and start parsing

In [5]:
from bs4 import BeautifulSoup
import requests

page = requests.get("http://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
today = forecast_items[0]
print(today.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Today
  <br>
   <br/>
  </br>
 </p>
 <p>
  <img alt="Today: Mostly cloudy, then gradually becoming sunny, with a high near 68. West southwest wind 8 to 13 mph increasing to 16 to 21 mph in the afternoon. Winds could gust as high as 26 mph. " class="forecast-icon" src="newimages/medium/bkn.png" title="Today: Mostly cloudy, then gradually becoming sunny, with a high near 68. West southwest wind 8 to 13 mph increasing to 16 to 21 mph in the afternoon. Winds could gust as high as 26 mph. "/>
 </p>
 <p class="short-desc">
  Decreasing
  <br>
   Clouds
  </br>
 </p>
 <p class="temp temp-high">
  High: 68 °F
 </p>
</div>


## Extracting information from the page

There are 4 pieces of information that can be extract:

    The name of the forecast item – in this case, Today.
    The description of the conditions – this is stored in the title property of img.
    A short description of the conditions – in this case, Mostly cloudy.
    The temperature high – in this case, 68 degrees.

Extracting the name of the forecast item, the short description, and the temperature first, since they’re all similar:

In [6]:
period = today.find(class_="period-name").get_text()
short_desc = today.find(class_="short-desc").get_text()
temp = today.find(class_="temp").get_text()

print(period)
print(short_desc)
print(temp)

Today
DecreasingClouds
High: 68 °F


Extracting the title attribute from the img tag.

In [7]:
img = today.find("img")
desc = img['title']

print(desc)

Today: Mostly cloudy, then gradually becoming sunny, with a high near 68. West southwest wind 8 to 13 mph increasing to 16 to 21 mph in the afternoon. Winds could gust as high as 26 mph. 


## Extracting all the information from the page

Extracting each individual piece of information

In [8]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['Today',
 'Tonight',
 'Thursday',
 'ThursdayNight',
 'Friday',
 'FridayNight',
 'Saturday',
 'SaturdayNight',
 'Sunday']

In [9]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]

print(short_descs)
print(temps)
print(descs)

['DecreasingClouds', 'Partly Cloudyand Breezythen MostlyCloudy', 'Mostly Sunny', 'Partly Cloudyand Breezythen PartlyCloudy', 'Mostly Sunny', 'Partly Cloudy', 'Sunny', 'Partly Cloudy', 'Mostly Sunny']
['High: 68 °F', 'Low: 57 °F', 'High: 68 °F', 'Low: 56 °F', 'High: 69 °F', 'Low: 56 °F', 'High: 70 °F', 'Low: 56 °F', 'High: 73 °F']
['Today: Mostly cloudy, then gradually becoming sunny, with a high near 68. West southwest wind 8 to 13 mph increasing to 16 to 21 mph in the afternoon. Winds could gust as high as 26 mph. ', 'Tonight: Partly cloudy, with a low around 57. Breezy, with a west southwest wind 17 to 22 mph decreasing to 11 to 16 mph after midnight. Winds could gust as high as 29 mph. ', 'Thursday: Mostly sunny, with a high near 68. West southwest wind 11 to 21 mph, with gusts as high as 26 mph. ', 'Thursday Night: Partly cloudy, with a low around 56. Breezy, with a west southwest wind 18 to 23 mph, with gusts as high as 30 mph. ', 'Friday: Mostly sunny, with a high near 69. West s

## Combining the data into a Pandas Dataframe

In [10]:
import pandas as pd
weather = pd.DataFrame({
        "period": periods, 
        "short_desc": short_descs, 
        "temp": temps, 
        "desc":descs
    })
weather

,desc,period,short_desc,temp
0,"Today: Mostly cloudy, then gradually becoming ...",Today,DecreasingClouds,High: 68 °F
1,"Tonight: Partly cloudy, with a low around 57. ...",Tonight,Partly Cloudyand Breezythen MostlyCloudy,Low: 57 °F
2,"Thursday: Mostly sunny, with a high near 68. W...",Thursday,Mostly Sunny,High: 68 °F
3,"Thursday Night: Partly cloudy, with a low arou...",ThursdayNight,Partly Cloudyand Breezythen PartlyCloudy,Low: 56 °F
4,"Friday: Mostly sunny, with a high near 69. Wes...",Friday,Mostly Sunny,High: 69 °F
5,"Friday Night: Partly cloudy, with a low around...",FridayNight,Partly Cloudy,Low: 56 °F
6,"Saturday: Sunny, with a high near 70.",Saturday,Sunny,High: 70 °F
7,"Saturday Night: Partly cloudy, with a low arou...",SaturdayNight,Partly Cloudy,Low: 56 °F
8,"Sunday: Mostly sunny, with a high near 73.",Sunday,Mostly Sunny,High: 73 °F


Pull out the numeric temperature values:

In [11]:
temp_nums = weather["temp"].str.extract("(?P<temp_num>\d+)", expand=False)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

0    68
1    57
2    68
3    56
4    69
5    56
6    70
7    56
8    73
Name: temp_num, dtype: object

Find the mean of all the high and low temperatures:

In [12]:
weather["temp_num"].mean()

63.666666666666664

Selecting the rows that happen at night:

In [14]:
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night
is_night

0    False
1     True
2    False
3     True
4    False
5     True
6    False
7     True
8    False
Name: temp, dtype: bool

In [15]:
weather[is_night]

,desc,period,short_desc,temp,temp_num,is_night
1,"Tonight: Partly cloudy, with a low around 57. ...",Tonight,Partly Cloudyand Breezythen MostlyCloudy,Low: 57 °F,57,True
3,"Thursday Night: Partly cloudy, with a low arou...",ThursdayNight,Partly Cloudyand Breezythen PartlyCloudy,Low: 56 °F,56,True
5,"Friday Night: Partly cloudy, with a low around...",FridayNight,Partly Cloudy,Low: 56 °F,56,True
7,"Saturday Night: Partly cloudy, with a low arou...",SaturdayNight,Partly Cloudy,Low: 56 °F,56,True
